In [1]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import cv2
import colorsys
import random
import pandas as pd

In [3]:
model = YOLO("/pfs/proj/nobackup/fs/projnb10/hpc2n2023-022/Yolo8-segmentation/runs/segment/train_x 643/weights/best.pt")  # load a custom model

img = cv2.imread("/pfs/stor10/users/home/s/sqbqamar/Public/Fiber_99_tiles/JPG/TileScan 1 Merged.jpg")

In [4]:
def detect(model, img):
        # Get img shape
    height, width, channels = img.shape
    results = model.predict(source=img.copy(), imgsz=6000, save=False, iou=0.8, conf=0.6, save_txt=False)
    result = results[0]
    segmentation_contours_idx = []
    for seg in result.masks.segments:
            # contours
        seg[:, 0] *= width
        seg[:, 1] *= height
        segment = np.array(seg, dtype=np.int32)
        segmentation_contours_idx.append(segment)

    bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        # Get class ids
    class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        # Get scores
    scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)
    #print(class_ids)
    return bboxes, class_ids, segmentation_contours_idx, scores

In [5]:
def draw_mask(img, pts, color, alpha=0.5):
    h, w, _ = img.shape

    overlay = img.copy()
    output = img.copy()

    cv2.fillPoly(overlay, pts, color)
    output = cv2.addWeighted(overlay, alpha, output, 1 - alpha,
                    0, output)
    return output

In [6]:
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 255 if bright else 180
    hsv = [(i / N + 1, 1, brightness) for i in range(N + 1)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors 

In [7]:
class_names = ['Fibre', 'Vessel']
colors = random_colors(len(class_names))
list2=[]

In [ ]:
bboxes, classes, segmentations, scores = detect(model, img)
for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
    # print("bbox:", bbox, "class id:", class_id, "seg:", seg, "score:", score)
    (x, y, x2, y2) = bbox
    rect = cv2.minAreaRect(seg)
    box1 = cv2.boxPoints(rect)
    box1 = np.int0(box1)
    #print(box1)
    a,b,c,d=box1
    dst1= ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5
    dst2 = ((b[0] - c[0])**2 + (b[1] - c[1])**2)**0.5
    dst1 = dst1 * 0.65
    dst2 = dst2 * 0.65
    if dst1 > dst2:
      Length = dst1
    else:
      Length = dst2
    area_px = cv2.contourArea(seg)
    area_px = area_px * 0.65
    Width = round(area_px/Length, 2)
    list1 = class_names[class_id], Length, Width, area_px
    list2.append(list1)
    
    #cv2.drawContours(img, [box1], 0, (0, 0, 255), 2)
    img = draw_mask(img, [seg], colors[class_id])
    #cv2.rectangle(img, (a, b), (c, d), (255, 0, 0), 2)
    cv2.drawContours(img,[box1],0,(255,0,0),2)


WARNING ⚠️ imgsz=[6000] must be multiple of max stride 32, updating to [6016]


In [ ]:
cv2.imwrite("/pfs/stor10/users/home/s/sqbqamar/Public/runs/segment/predict/Predicted_TileScan 1 Merged1.jpg", img)

"""
df = pd.DataFrame(list2)
df1 = df.rename(columns={0: 'Class Name',1: 'Length', 2: 'Width', 3: 'Area'})
df1.to_excel("/pfs/stor10/users/home/s/sqbqamar/Public/Fiber_99_tiles/Result_micro/image_3.xlsx")
"""
